# AI Driver Management

In [2]:
import os
import csv
import cv2
import dlib
import socket
import radar
import urllib.request
import numpy as np
import pandas as pd
import face_recognition as fr
from requests import get
from radar import RadarClient
from imutils import face_utils
from datetime import datetime
from geopy.geocoders import Nominatim

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
key_file = open("radar_api/key.txt","r")
api_key = key_file.read()

In [5]:
path = 'Face Data'
faces = os.listdir("Face Data")
encodings = []
face_names = []
for face in faces :
    to_test = fr.load_image_file(f'{path}/{face}')
    to_test = cv2.cvtColor(to_test, cv2.COLOR_BGR2RGB)
    encoding = fr.face_encodings(to_test)[0]
    encodings.append(encoding)
    face_names.append(face.split(".")[0])

In [6]:
df = pd.DataFrame(columns=['Name', 'Face-In', 'Face-Out', 'Duration', 'Origin', 'Destination'])
df['Name']= face_names

In [7]:
def recognizer(frames, face_names, face_encodings) :
    frames_resized = cv2.resize(frames, (0,0), None, 0.20, 0.20)
    frames_resized = cv2.cvtColor(frames_resized, cv2.COLOR_BGR2RGB)
    loc_in_frame = fr.face_locations(frames_resized)
    encodings_in_frame = fr.face_encodings(frames_resized, loc_in_frame)
    
    for face_encoding in encodings_in_frame :
        face_distance = fr.face_distance(face_encodings, face_encoding)
        return np.argmin(face_distance)

In [8]:
def get_ip_location():
    radar= RadarClient(api_key)
    ip_address = urllib.request.urlopen('https://ident.me').read().decode('utf8') 
    ip_location = radar.geocode.ip(ip=ip_address)
    
    return ip_location 

In [9]:
def distance_matrix(destination_address, origin_address=get_ip_location().city):
    loc = Nominatim(user_agent="GetLoc")
    radar = RadarClient(api_key)
    
    origin = loc.geocode(origin_address)
    origin = (origin.latitude, origin.longitude)
    destination = loc.geocode(destination_address)
    destination = (destination.latitude, destination.longitude)
    
    routes = radar.route.distance(origin, destination, modes="car")
    return routes.car

In [10]:
def calc_distances(f_id_list, data) :
    for i in f_id_list :
        data.loc[i, "Distance Travelled"] = distance_matrix(data.loc[i, "Origin"], data.loc[i, "Destination"]).distance.text 
        
    return data

In [11]:
def get_fuel_prices(city) :
    geolocator = Nominatim(user_agent = "http")
    loc = geolocator.geocode(city)
    state = loc[0].split(",")
    
    link_table = pd.read_csv("links/fuel_price_links.csv")
    link_table['States']=link_table.iloc[:,1]
    state_found = list(set(list(link_table['States'])).intersection(set(state)))[0]
    
    if state_found == None :
        print("City not in database.")
        return
    
    link = str(link_table[link_table['States']==state_found]['Links']).split('\n')[0]
    link = "https://www.goodreturns.in/"+str(link).split("/")[1]
    link = link.rstrip("\n")
    print(link)
    html_table = pd.read_html(link)
    ht = html_table[0]
    ht.columns = ht.iloc[0]
    ht = ht[1:]
    ht.loc[:, ["Today Price", 
               "Yesterday Price"]]=ht[ht.columns[1:]].replace('[\₹,]', '', regex=True).astype(float)
    display(ht)
    return ht["Today Price"].mean()

In [12]:
def calc_fare(f_id_list ,data, average):
    for i in f_id_list :
        if str(data['Distance Travelled'].dtype)=='object':
            data['Distance Travelled']=data['Distance Travelled'].str.replace(r'km', '').astype(float)
        data.loc[i, 'Fare'] = (get_fuel_prices(data.loc[i, 'Origin'])*data.loc[i, 'Distance Travelled'])/average
    return data['Fare']

In [13]:
webcam = cv2.VideoCapture(0)
i=0
f_id_list = []
dtime, dwell_time, face_in, face_out, origin_dict, destination_dict = [dict() for _ in range(6)]

while True :
    if i%200 == 0:
        success , frames = webcam.read()
        f_id = recognizer(frames, face_names, encodings)
        if f_id!=None :
            cv2.putText(frames, 
                        face_names[f_id] ,(50, 50) , 
                        cv2.FONT_HERSHEY_COMPLEX,1.2, (0, 255, 255), 1)

            if f_id not in f_id_list :
                f_id_list.append(f_id)
                dtime[f_id] = datetime.now()
                dwell_time[f_id] = 0
                face_in[f_id] = datetime.now()
                origin_dict[f_id] = get_ip_location().city 
            else :
                curr_time = datetime.now()
                old_time = dtime[f_id]
                time_diff = curr_time - old_time
                dtime[f_id] = datetime.now()
                sec = time_diff.total_seconds()
                dwell_time[f_id] += sec
                face_out[f_id] = datetime.now()
                #destination_dict = get_ip_location().city
                #The above line is commented for demonstration purposes, as it will return same 
                #address
                
        
        cv2.imshow("recognizer", frames)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            del webcam
            break
    i+=1

In [14]:
destination_dict = {0:"Mumbai", 1:"Delhi", 2:"Bangalore", 3:"Jaipur"}

In [15]:
for i in f_id_list :
    df.loc[i, "Face-In"] = face_in[i].strftime("%H:%M:%S")
    df.loc[i, "Face-Out"] = face_out[i].strftime("%H:%M:%S")
    df.loc[i, "Duration"] = dwell_time[i]
    df.loc[i, "Origin"] = origin_dict[i]
    df.loc[i, "Destination"] = destination_dict[i]

In [16]:
df = calc_distances(f_id_list, df)

In [17]:
df['Fare']=calc_fare(f_id_list, df, 20)

https://www.goodreturns.in/petrol-price-in-gujarat-s12.html


,City,Today Price,Yesterday Price
1,Ahmedabad,95.13,95.13
2,Amreli,96.63,96.71
3,Anand,95.26,95.26
4,Aravalli,95.89,95.73
5,Banas Kantha,95.33,94.92
6,Bharuch,95.7,95.7
7,Bhavnagar,97.62,96.5
8,Botad,96.39,96.39
9,Chhotaudepur,95.12,95.12
10,Dahod,96.23,95.69


In [18]:
df

,Name,Face-In,Face-Out,Duration,Origin,Destination,Distance Travelled,Fare
0,disaster_girl,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pragnesh,17:37:10,17:37:20,10.163109,Rajkot,Delhi,1140.5,5456.013758
2,tony_stark,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,xavier,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
A[1].append(4)